# Download data from basespace

In [ ]:
PROJECT="{{ project }}"
PATTERN="{{ pattern }}"
RUNID="{{ runid }}"
NPROC="{{ num_proc }}"
BSDIR="{{ bs_dir }}"
base_dir="{{ base_dir }}"

# Trimming parameters
clipfile="{{ clip_file }}"
winlength="{{ win_length }}"
winqual="{{ win_qual }}"
minlength="{{ min_length }}"

# Local stuff
base_dir="../data"
trimmer=$(which TrimmomaticPE)

In [ ]:
basedir="$BSDIR/Projects/$PROJECT/Samples"

for sample in "$basedir/$RUNID"*
do
    sampleid=$(echo $sample | cut -d '-' -f 3)
    mkdir -p ../data/$sampleid
    echo "Copying $sample into ../data/$sampleid"
    cp $sample/Files/*.fastq.gz ../data/$sampleid
done


In [ ]:
ls -lah ../data/*

# Merge lanes in each sample

In [ ]:
for sample in $(ls $base_dir | grep "$PATTERN")
do
    sample_dir=$base_dir/$sample
    echo "Merging lanes in" $sample_dir
    
    cat $sample_dir/*R1_001* > $sample_dir/R1.fastq.gz
    cat $sample_dir/*R2_001* > $sample_dir/R2.fastq.gz
done

## Cleanup

In [ ]:
for sample in $(ls $base_dir | grep "$PATTERN")
do
    sample_dir=$base_dir/$sample
    echo "Merging lanes in" $sample_dir
    
    rm -rf $sample_dir/ig*.fastq.gz
    ls -lah $sample_dir
done

## Quality trimming

In [ ]:
for sample in $(ls $base_dir | grep "$PATTERN")
do
    sample_dir=../data/$sample
    trim_dir=$sample_dir/trim
    
    if [ ! -d "$trim_dir" ]; then
        echo "Creating $trim_dir ..."
        mkdir $trim_dir
    fi
    
    $trimmer -threads $NPROC $sample_dir/R1.fastq.gz $sample_dir/R2.fastq.gz \
        $trim_dir/R1_pe.fastq.gz $trim_dir/R1_se.fastq.gz \
        $trim_dir/R2_pe.fastq.gz $trim_dir/R2_se.fastq.gz \
        ILLUMINACLIP:$clipfile:2:30:10 \
        SLIDINGWINDOW:$winlength:$winqual MINLEN:$minlength

done